The following Python script was developed to automate the retrieval of historical air quality and meteorological observations from the NSW Government Air Quality API. The structure and request format of this script were taken directly from the official API documentation and sample code provided on the NSW Government’s website (NSW Government, 2024a; 2024b, mentioned in Part A report). This ensures compliance with the approved data-access methodology and guarantees that the dataset used in this study is consistent with official standards.   
**Functionality and Purpose**  

- **API Class (aqms_api_class):** Configures the NSW Air Quality API with the correct URL, headers, and endpoints as specified by the official documentation.

- **Request Builder (ObsRequest_init):** Constructs a valid POST request to retrieve pollutants (PM10, PM2.5, NO₂, CO, O₃) and meteorological variables (wind speed, wind direction, sigma theta, temperature, humidity, solar radiation, rainfall) across five monitoring sites in Sydney (Rozelle, Chullora, Parramatta North, Campbelltown West, and Liverpool)
.

- **Temporal Coverage:** Illustrates retrieval of a one-month dataset (June 2015), which serves as a proof-of-concept for scaling to the full ten-year range.

- **Output:** The raw JSON response is saved locally as HistoricalObs_2015_June.txt, providing a reproducible record before conversion to CSV and further preprocessing.

In [ ]:
import pandas as pd
import os
import sys
import requests
import logging
import urllib
import json
import datetime as dt

In [ ]:


#####################################################################
class aqms_api_class(object):
    """
    This class defines and configures the API to query the aqms database
    """
    def __init__(self):
        self.logger = logging.getLogger("aqms_logger")
        self.url_api = "https://data.airquality.nsw.gov.au"
        self.headers = {'content-type': 'application/json', 'accept': 'application/json'}
        self.get_observations = 'api/Data/get_Observations'

    def get_Obs(self, ObsRequest):
        '''
        Send POST request to return observation details
        '''
        query = urllib.parse.urljoin(self.url_api, self.get_observations)
        response = requests.post(url=query, data=json.dumps(ObsRequest), headers=self.headers)
        return response

#####################################################################
def ObsRequest_init():
    '''
    Build a query to return all historical observations
    '''
    ObsRequest = {}
    ObsRequest['Parameters'] = [
        'PM10', 'PM2.5', 'NO2', 'CO', 'O3',    # pollutants
        'WS', 'WD', 'SD1', 'TEMP', 'RH', 'SR', 'RAIN'  # meteorological
    ]
    ObsRequest['Sites'] = [39, 1141, 919, 2560, 107]  # Site IDs
    StartDate = dt.date(2015, 6, 1)
    EndDate = dt.date(2015, 7, 1)
    ObsRequest['StartDate'] = StartDate.strftime('%Y-%m-%d')
    ObsRequest['EndDate'] = EndDate.strftime('%Y-%m-%d')
    ObsRequest['Categories'] = ['Averages']
    ObsRequest['SubCategories'] = ['Hourly']
    ObsRequest['Frequency'] = ['Hourly average']
    return ObsRequest

#####################################################################
if __name__ == '__main__':
    AQMS = aqms_api_class()
    ObsRequest = ObsRequest_init()
    AllHistoricalObs = AQMS.get_Obs(ObsRequest)

    # Save Historical Observations to a text file
    with open('HistoricalObs_2015_June.txt', 'w', encoding='utf-8') as f:
        f.write(AllHistoricalObs.text)


In [ ]:

#Reading the dataset file
with open('HistoricalObs_2015_June.txt', 'r', encoding='utf-8') as f:
    data = json.load(f)  # JSON string → Python list/dict

# JSON to pandas DataFrame
df = pd.json_normalize(data)

# Converting to CSV
df.to_csv('HistoricalObs_2015_June.csv', index=False)

print("CSV file saved as 'HistoricalObs_2015_June.csv'")


CSV file saved as 'HistoricalObs_2015_June.csv'


In [50]:
df = pd.read_csv("HistoricalObs_2015_June.csv")
df.head(20)

,Site_Id,Date,Hour,HourDescription,Value,AirQualityCategory,DeterminingPollutant,Parameter.ParameterCode,Parameter.ParameterDescription,Parameter.Units,Parameter.UnitsDescription,Parameter.Category,Parameter.SubCategory,Parameter.Frequency
0,39,2015-06-01,1,12 am - 1 am,0.275744,NaN,NaN,CO,Carbon monoxide,ppm,parts per million,Averages,Hourly,Hourly average
1,39,2015-06-01,1,12 am - 1 am,1.140613,GOOD,NaN,NO2,Nitrogen Dioxide,pphm,parts per hundred million,Averages,Hourly,Hourly average
2,39,2015-06-01,1,12 am - 1 am,6.484000,GOOD,NaN,PM10,PM10,µg/m³,microgram per cubic meter,Averages,Hourly,Hourly average
3,39,2015-06-01,1,12 am - 1 am,8.034000,GOOD,NaN,PM2.5,PM2.5,µg/m³,microgram per cubic meter,Averages,Hourly,Hourly average
4,39,2015-06-01,1,12 am - 1 am,NaN,NaN,NaN,RAIN,Rainfall,mm/m²,millimetre rainfall,Averages,Hourly,Hourly average
5,39,2015-06-01,1,12 am - 1 am,49.993000,NaN,NaN,SD1,Wind Direction Sigma Theta,°,degree,Averages,Hourly,Hourly average
6,39,2015-06-01,1,12 am - 1 am,9.700000,NaN,NaN,TEMP,Temperature,°C,degree Celsius,Averages,Hourly,Hourly average
7,107,2015-06-01,1,12 am - 1 am,0.212361,NaN,NaN,CO,Carbon monoxide,ppm,parts per million,Averages,Hourly,Hourly average
8,107,2015-06-01,1,12 am - 1 am,1.212620,GOOD,NaN,NO2,Nitrogen Dioxide,pphm,parts per hundred million,Averages,Hourly,Hourly average
9,107,2015-06-01,1,12 am - 1 am,6.395000,GOOD,NaN,PM10,PM10,µg/m³,microgram per cubic meter,Averages,Hourly,Hourly average


In [ ]:



df = pd.read_csv("HistoricalObs_2015_June.csv")  

# Convert Date to datetime 
df['ParsedDate'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')

# Extract start hour string from HourDesc
df['StartHourStr'] = df['HourDescription'].str.extract(r'(^[\d]+ ?[ap]m)', expand=False)

# Start hour to integer (24-hour format)
df['HourInt'] = pd.to_datetime(df['StartHourStr'], format='%I %p', errors='coerce').dt.hour

# Generating full timestamp with date and hour
df['Timestamp'] = df['ParsedDate'] + pd.to_timedelta(df['HourInt'], unit='h')

# Dropping rows where Timestamp could not be formed
df = df.dropna(subset=['Timestamp'])

# Pivot the data
pivoted_df = df.pivot_table(
    index=['Site_Id', 'Timestamp'],
    columns='Parameter.ParameterCode',
    values='Value',
    aggfunc='first'
).reset_index()

pivoted_df.columns.name = None

# Exporting to CSV
pivoted_df.to_csv("transformed_output.csv", index=False)


In [96]:
df=pd.read_csv("transformed_output.csv")

In [98]:
df.head()

,Site_Id,Timestamp,CO,NO2,PM10,PM2.5,SD1,TEMP
0,39,2015-01-06 00:00:00,0.275744,1.140613,6.484,8.034,49.993,9.700
1,39,2015-01-06 01:00:00,NaN,NaN,0.477,4.407,27.682,11.954
2,39,2015-01-06 02:00:00,0.076486,0.093000,5.542,3.822,21.372,12.436
3,39,2015-01-06 03:00:00,0.087004,0.235300,3.896,2.004,20.994,11.786
4,39,2015-01-06 04:00:00,0.078759,0.375000,7.401,3.383,21.597,11.968
